In [1]:
import pycdfpp
import cdflib
from spacepy import pycdf
import timeit
import matplotlib.pyplot as plt 
import requests
import os

In [2]:
%%bash
mkdir -p /tmp/cdfpp
curl https://hephaistos.lpp.polytechnique.fr/data/mirrors/CDF/test_files/po_h9_pwi_1997010103_v01.cdf > /tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  304M  100  304M    0     0   111M      0  0:00:02  0:00:02 --:--:--  111M


In [2]:
def cdflib_just_open(fname):
    c = cdflib.CDF(fname)

def spacepy_just_open(fname):
    c = pycdf.CDF(fname)

def pycdfpp_just_open(fname):
    c = pycdfpp.load(fname)

In [3]:
timeit.timeit(lambda: pycdfpp_just_open('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

1.0989350419258699

In [4]:
timeit.timeit(lambda: cdflib_just_open('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

0.3326520000118762

In [5]:
timeit.timeit(lambda: spacepy_just_open('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

5.858953066985123

In [6]:
def cdflib_list_variables(fname):
    c = cdflib.CDF(fname)
    cdf_info = c.cdf_info()
    variables = cdf_info.rVariables + cdf_info.zVariables

def spacepy_list_variables(fname):
    variables = list(pycdf.CDF(fname))

def pycdfpp_list_variables(fname):
    variables = list(pycdfpp.load(fname))
    

In [7]:
timeit.timeit(lambda: pycdfpp_list_variables('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

1.2540769890183583

In [8]:
timeit.timeit(lambda: cdflib_list_variables('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

1.7590590990148485

In [9]:
timeit.timeit(lambda: spacepy_list_variables('/tmp/cdfpp/po_h9_pwi_1997010103_v01.cdf'), number=10000)

7.060896105016582